In [78]:
import os, csv, pandas as pd, string
from nltk.metrics import edit_distance as ed

In [217]:
word_dict = {
    'the': 1,
    'beautiful': 2,
    'ducks': 3,
    'swam': 4,
    'in': 5,
    'big': 6,
    'blue': 7,
    'lake': 8,
    'their': 9,
    'feathers': 10,
    'were': 11,
    'very': 12,
    'pretty': 13,
    'and': 14,
    'quite': 15,
    'often': 16,
    'would': 17,
    'jump': 18,
    'out': 19,
    'of': 20,
    'water': 21,
}
word_dict_key = list(word_dict.keys())
sim_con = {
    'b': ['d','h','q','p'],
    'd': ['b', 'h', 'q','p'],
    'h': ['b', 'd', 'q', 'p'],
    'q': ['b', 'd', 'h', 'p'],
    'p': ['b', 'd', 'h', 'q'],
    'f': ['t'],
    't': ['f'],
}
sim_vow = {
    'o': ['u', 'e'],
    'i': ['e'],
    'a': ['e'],
    'e': ['i', 'a', 'u']
}
consonants = ['b', 'c', 'd', 'f', 'g', 'h', 'j', 'k', 'l', 'm', 'n', 'p', 'q', 'r', 's', 't', 'v', 'w', 'x', 'y', 'z']
vowels = ['a', 'e', 'i', 'o', 'u']
# print(word_dict_key)
def sim_characters(src, tgt):
    score = []
    phe = 0
    unv = 0
    aee = 0
    doc = 0
    # print(src, tgt)

    # if length of src and tgt is same but the error is in character
    if len(src) == len(tgt):
        for c in range(len(src)):

            if src[c] != tgt[c]:
                # print('here')
                # phonetic error [6]
                if src[c] in sim_con.keys() and tgt[c] in sim_con[src[c]]:
                    # print('here')
                    phe += 1

                #unstressed vowel [10]
                elif src[c] in sim_vow.keys() and tgt[c] in sim_vow[src[c]]:
                    unv += 1
                    # print('here')
        # score.extend[phe, unv]


    # if length of tgt is greater, meaning the child has written extra character
    elif len(src) < len(tgt):
        
        prev_c = ''
        for c in range(len(tgt)):

            # adding extra e [1]
            if c == len(tgt) - 1:
                if tgt[-1] == 'e' and src[-1] != 'e': aee += 1

            if c < len(src):
                #phonetic error [6]
                if src[c] in sim_con.keys() and tgt[c] in sim_con[src[c]]: phe += 1
                
                #doubling of consonants [3]
                if src[c] != tgt[c] and tgt[c] == prev_c and tgt[c] in consonants: doc += 1
                elif src[c] == tgt[c]: prev_c = src[c]
        # score.extend([phe, unv, aee, doc])
            

    return phe, unv, aee, doc

def scoring(df):

    sentence_length = list()
    PHE = []    #phonetic error [6]
    UNV = []    #unstressed vowel [10]
    AEE = []    #added extra e [1]
    DOC = []    #doubling of consonants [3]

    for ind, row in df.iterrows():
        phe, unv, aee, doc = 0, 0, 0, 0
        words = row['sentence'].translate(str.maketrans('','',string.punctuation)).lower()
        words = words.split()
        # score = []
        #length of sentence
        if len(words) == 25: sentence_length.append(0)
        else: sentence_length.append(len(words))

        # print(word_dict_key[0], words[0])

        for word in words:
            src = ''
            tgt = word
            min_dist = 999

            # edit distance to find the closest word if there are spelling errors
            for x in word_dict_key:
                dist = ed(word, x)
                if dist < min_dist:
                    min_dist = dist
                    src = x
            if min_dist > 0:    
                a, b, c, d = sim_characters(src, tgt)
                phe += a
                unv += b
                aee += c
                doc += d
            
        PHE.append(phe)
        UNV.append(unv)
        AEE.append(aee)
        DOC.append(doc)
            # score += sim_characters(src, tgt)
            # print(min_dist, src, tgt, score)
            # break
        # break

    df['sentence length'] = sentence_length
    df['phonetic error'] = PHE
    df['unstressed vowel'] = UNV
    df['adding extra e'] = AEE
    df['doubling consonant'] = DOC
    
    return df

In [218]:
#1: there is error
#0: no error

df = pd.read_csv('data.csv')
df = scoring(df)
# print(len(df['id']))
df.to_csv('data.csv', index=False)
print(df)


              id                                           sentence  \
0         sample  Tbe beauttiful dacks swame in the big blue lak...   
1           30_1  The beautiful ducks swam in the big blue lake....   
2           30_5  The fish is not happy because it is in a small...   
3           32_3  cat's are predetors they drinkmilk and is afra...   
4           33_3  The dog is happy.The kitten is sleeping. The f...   
5        34-nt_1  The dog is eager to eat Food. The cat is in a ...   
6        35-nt_3  The bird is squaking happily on a stick. The f...   
7         DIY001  The beautiful ducks swam in the big blue lake....   
8         DIY002  The beautiful ducks swam in the big blue lake....   
9         MAR001  The beautiful ducks swam in the big blue lake....   
10        MAR002  The beautiful ducks swam in the big blue lake....   
11      MAR003.2  The beautful ducks swam in the big blue lake. ...   
12        MAR003  The beautiful ducks sawm inth big bluelake. Th...   
13    